# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [3]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [4]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [5]:
# Importing the necessary CSVs to Pandas DataFrames
# Original file path.
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
# DataFrames for Housing Units Per Year plot.
sfo_data = pd.read_csv(file_path, index_col="year")
housing_units = sfo_data[['housing_units']]
housing_units_mean = housing_units.groupby('year').mean()
housing_units_mean.to_csv('housing_units_mean_file.csv')
file_path1 = Path("housing_units_mean_file.csv")
housing_units_mean_df = pd.read_csv(file_path1, index_col="year")
housing_units_mean_df = housing_units_mean_df.rename(columns={"housing_units" : "Housing Units"})
# DataFrames for Average Housing Costs in SF per year plot.
sales_price_gross_rent = sfo_data[['sale_price_sqr_foot', 'gross_rent']]
sales_price_gross_rent_avg = sales_price_gross_rent.groupby('year').mean()
sales_price_gross_rent_avg = sales_price_gross_rent_avg.rename(columns={"sale_price_sqr_foot" : "Price per SqFt", "gross_rent" : "Gross Rent" })
sales_price_df = sales_price_gross_rent_avg[['Price per SqFt']]
gross_rent_df = sales_price_gross_rent_avg[['Gross Rent']]
# DataFrames for Average Prices by Neighborhood.
sfo_data_mean_values = pd.DataFrame(
    sfo_data.groupby(
        ['year', 'neighborhood'])[['sale_price_sqr_foot', 'housing_units', 'gross_rent']].mean()
        ).reset_index()
sfo_data_mean_values = sfo_data_mean_values.rename(
    columns={"year" : "Year",
             "neighborhood" : "Neighborhood",
             "sale_price_sqr_foot" : "Price per SqFt",
             "housing_units": "Housing Units",
             "gross_rent" : "Gross Rent" })
# DataFrames for Top 10 Most Expensive Neighborhood plot.
sfo_data_sales_price_avg = sfo_data.groupby('neighborhood').mean().reset_index()
sfo_top = sfo_data_sales_price_avg.sort_values('sale_price_sqr_foot', ascending=False)
sfo_top_ten = sfo_top.iloc[:10, 0:2]
sfo_top_ten_df = (
    sfo_top_ten
    .rename(
        columns={
            "neighborhood" : "Neighborhood",
            "sale_price_sqr_foot" : "Price per SqFt"
        })
    .set_index('Neighborhood')
)
# DataFrames for Neighborhood Map
neighborhood_coordinates_df = pd.read_csv(Path('./Data/neighborhoods_coordinates.csv'))
neighborhood_coordinates_df = neighborhood_coordinates_df.set_index("Neighborhood")
neighborhood_df_mean = sfo_data_mean_values.groupby('Neighborhood').mean()
neighborhood_df_mean = neighborhood_df_mean.drop(columns=['Year'])
neighborhood_joined = pd.concat([neighborhood_coordinates_df, neighborhood_df_mean], join='inner', axis=1)
# Top 10 Most Expensive SF Neighborhoods DataFrame for:
# - Parallel Categories Plot,
# - Parallel Coordinates Plot,
# - Sunburst Plot
top_10 = sfo_data_mean_values.sort_values('Price per SqFt', ascending=False).head(10)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [11]:
# Defining Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    return housing_units_mean_df.hvplot.bar(
            ylim=[housing_units_mean_df["Housing Units"].min() - housing_units_mean_df["Housing Units"].std(),
              housing_units_mean_df["Housing Units"].max() + housing_units_mean_df["Housing Units"].std()],
            rot=90).opts(title='Housing Units in San Francisco from 2010 to 2016', yformatter='%.0f')
    

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    return gross_rent_df.hvplot().opts(title='Average Gross Rent by Year', yformatter='%.0f', line_color='purple')


def average_sales_price():
    """Average Sales Price Per Year."""
    return sales_price_df.hvplot().opts(title='Average Price per SqFt by Year', yformatter='%.0f', line_color='green')


def average_price_by_neighborhood_per_sqft():
    """Average Prices by Neighborhood."""
    return sfo_data_mean_values.hvplot(
        x="Year",
        y="Price per SqFt",
        title="Neighborhood",
        groupby="Neighborhood"
    )


def average_price_by_neighborhood_by_gross_rate():
    """Average Prices by Neighborhood."""
    return sfo_data_mean_values.hvplot(
        x="Year",
        y="Gross Rent",
        title="Neighborhood",
        groupby="Neighborhood"
    )


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    return sfo_top_ten_df.hvplot.bar(rot=90).opts(title='Top 10 Expensive Neighborhoods in SFO', yformatter='%.0f')


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    return sfo_data_mean_values.hvplot.bar(
        x="Year",
        y=["Price per SqFt", "Gross Rent"], 
        title="Neighborhood",
        groupby="Neighborhood",
        rot=90
    )
    
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    return px.parallel_coordinates(
        top_10,
        dimensions=["Neighborhood", "Price per SqFt", "Housing Units", "Gross Rent"],
        color_continuous_scale=px.colors.sequential.Inferno,
        color="Price per SqFt"
    )


def parallel_categories():
    """Parallel Categories Plot."""
    return px.parallel_categories(
        top_10,
        dimensions=["Neighborhood", "Price per SqFt", "Housing Units", "Gross Rent"],
        color_continuous_scale=px.colors.sequential.Inferno,
        color="Price per SqFt"
    )


def neighborhood_map():
    """Neighborhood Map."""
    return px.scatter_mapbox(
    neighborhood_joined,
    lat="Lat",
    lon="Lon",
    color=neighborhood_joined.index,
    zoom=10
)


#def sunburst():
#    """Sunburst Plot."""
    
    # YOUR CODE HERE!


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [14]:
# Create a Title for the Dashboard
# YOUR CODE HERE!


# Create a tab layout for the dashboard
housing_units_per_year_tab = pn.Column(
    "## Housing Units in Saf Francisco Per Year",
    housing_units_per_year()
)

average_tab = pn.Column(
    "## Average Sale Price per Square Foot, Monthly Rent Charts, and Prices per Neighborhood",
    average_gross_rent(),
    average_sales_price(),
    average_price_by_neighborhood_per_sqft(),
    average_price_by_neighborhood_by_gross_rate()
)

top_ten_tab = pn.Column(
    "## Top 10 Most Expensive Neighborhoods",
    top_most_expensive_neighborhoods()
)

cost_vs_income_tab = pn.Column(
    "## Comparison of Cost to Purchase versus Rental Income",
    most_expensive_neighborhoods_rent_sales()
)

parallel_plots_tab = pn.Column(
    "## Parallel Coordinates and Parallel Categories",
    parallel_coordinates(),
    parallel_categories()
)

neighborhood_map_tab = pn.Column(
    "## Neighborhood Map",
    neighborhood_map()
)

# Create the dashboard
sfo_neighborhoods_dashboard = pn.Tabs(
    (
        "Units per Year",
        housing_units_per_year_tab
    ),
    (
        "Average",
        average_tab
    ),
    (
        "Top Ten",
        top_ten_tab
    ),
    (
        "Cost vs Income",
        cost_vs_income_tab    
    ),
    (
        "Parallels",
        parallel_plots_tab
    ),
    (
        "Geospatial",
        neighborhood_map_tab
    )

)

## Serve the Panel Dashboard

In [15]:
# Serve the# dashboard
sfo_neighborhoods_dashboard

Tabs
    [0] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
        [3] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='Neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [4] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='Neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [2] Column
        [0] Markdown(str)
        [1] HoloViews(Bars)
    [3] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='Neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
    [4] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [5] Column
        [0] Markdown(str)
        [1] Plotly(Figure)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()